**Analyze the accesibility of hospitals in Boston by considering the nearby sidewalks (Ramp scores)**

Generate the Hospital_Scores5.html

In [1]:

import folium
import json
import geojson
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, Point, Polygon


ramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
ramps_gdf = ramps_gdf.to_crs(sidewalks_gdf.crs)
ramps_gdf.geometry = ramps_gdf.geometry.buffer(5)


# print(sidewalks_gdf.head())
# print(ramps_gdf.head())



In [2]:
# Joining the two dataframes. Give scores to the ramps based on the number of ramps in the buffer
sidewalks_gdf = sidewalks_gdf[sidewalks_gdf.geometry.notna()]
ramps_gdf = ramps_gdf[ramps_gdf.geometry.notna()]

joined_gdf = gpd.sjoin(sidewalks_gdf, ramps_gdf, op='intersects')
print(joined_gdf.head())
grouped_gdf = joined_gdf.groupby('SWK_ID').agg({'FID_left': list, 'score_1': 'mean'}).reset_index()
result_gdf = sidewalks_gdf.merge(grouped_gdf, on='SWK_ID', how='left')
print(result_gdf.head())



c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


      FID_left SWK_ID MATERIAL SWK_WIDTH          DISTRICT SWK_AREA  PARENT  \
0            0      1       CC       5.5       CHARLESTOWN     3648  ALBIO1   
9186      9186   9844       CC        17       CHARLESTOWN     3289  MAIN 2   
1            1    402       CC         6  SOUTH DORCHESTER     1671  ALMON1   
2            2    403       CC         6  SOUTH DORCHESTER     1351  ALMON1   
7            7    408       CC         6  SOUTH DORCHESTER     1445  ALABA2   

           SEG_ID   SIDE  CG_ID  ...  \
0        ALBIO1_0   LEFT  24842  ...   
9186  MAIN 2_3764   LEFT   9784  ...   
1     ALMON1_1749   LEFT    401  ...   
2     ALMON1_2047  RIGHT    402  ...   
7        ALABA2_0  RIGHT    407  ...   

                                                  Photo  NEAR_FID  NEAR_DIST  \
0     V:\1794\active\179410852\CREEC Ramps\CREEC_QC_...        54  16.117564   
9186  V:\1794\active\179410852\CREEC Ramps\CREEC_QC_...        54  16.117564   
1     V:\1794\active\179410852\CREEC Ramps\C

In [3]:
new_gdf = result_gdf

Introduce the informations of hospitals

In [4]:
m = folium.Map(location=[42.361145, -71.057083], zoom_start=30)

hospitals_gdf = gpd.read_file('../data/Hospitals.geojson')
sidewalk_gdf = new_gdf.to_crs(hospitals_gdf.crs)
sidewalk_gdf = sidewalk_gdf.drop(columns=['FID_left'])

print(hospitals_gdf)

    OBJECTID  Census_Tra                                               Name  \
0          1           0                                    Arbour Hospital   
1          2           0      Franciscan Children's Hospital & Rehab Center   
2          3           0                       Hebrew Rehabilitation Center   
3          4           0         Jewish Memorial Hospital & Radius Hospital   
4          5           0                                   Kindred Hospital   
5          6           0                           Lemuel Shattuck Hospital   
6          7           0                          Shriner’s Burn Institute    
7          8           0                  Spaulding Rehabilitation Hospital   
8          9        2069  Beth Isreal Deaconess medical Center - East Ca...   
9         10        2307              Boston Medical Center - Menino Campus   
10        11        2341                       Brigham and Women's Hospital   
11        12        2003                            

In [5]:

# print(joined_gdf.columns)

# Create a buffer around hospitals
buffer_distance = 0.005
hospitals_gdf['geometry'] = hospitals_gdf.geometry.buffer(buffer_distance)

#  Join the sidewalks with the hospitals
joined_gdf = gpd.sjoin(sidewalk_gdf, hospitals_gdf, how='left', op='within')
# print(joined_gdf.head())
# print(joined_gdf.columns)

# Calculate the average score for each hospital
hospital_scores = joined_gdf.groupby(['FID','Name'])['score_1'].mean().reset_index()
hospital_scores = hospital_scores.rename(columns={'FID': 'FID'})
# print(hospital_scores)
# print(hospitals_gdf.head())
hospitals_with_scores_gdf = hospitals_gdf.merge(hospital_scores, on='Name', how='left')


hospitals_with_scores_gdf = hospitals_with_scores_gdf.dropna(subset=['score_1'])

#  Drop duplicates
print(hospitals_with_scores_gdf.columns)
avg_scores = hospitals_with_scores_gdf.groupby('Name')['score_1'].mean().reset_index()
hospitals_with_scores_gdf = hospitals_with_scores_gdf.drop_duplicates(subset=['FID'])
hospitals_with_scores_gdf = hospitals_with_scores_gdf.drop_duplicates(subset=['Name']).merge(avg_scores, on='Name', suffixes=('', '_avg'))
hospitals_with_scores_gdf = hospitals_with_scores_gdf.drop(columns=['score_1']).rename(columns={'score_1_avg': 'score_1'})




# print(hospitals_with_scores_gdf)
hospitals_with_scores_gdf.to_file('../data/HospitalsBufferScore.geojson', driver='GeoJSON')


Index(['OBJECTID', 'Census_Tra', 'Name', 'Address', 'City', 'Zipcode',
       'Statea', 'Contact', 'PhoneNumbe', 'Latitude', 'Longitude', 'Comment_',
       'Primary_Alt', 'Alternative_Name', 'DailyAvg', 'geometry', 'FID',
       'score_1'],
      dtype='object')


C:\Users\xiao\AppData\Local\Temp\ipykernel_25288\277478078.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hospitals_gdf['geometry'] = hospitals_gdf.geometry.buffer(buffer_distance)
c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Add the dataframes to the map

In [6]:
# Add hospitals layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=hospitals_with_scores_gdf,
    name='Hospital Colors',
    data=hospitals_with_scores_gdf,
    columns=['FID', 'score_1', 'Name'],
    key_on='feature.properties.FID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='Average Score',
    bins=list(range(0, 101, 10)),
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

print(hospitals_with_scores_gdf['score_1'].isna().sum())

# Adds the ability to click the hospital to see the score for that hospital
folium.GeoJson(
    hospitals_with_scores_gdf,
    name='Hospital Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID', 'Name'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1', 'Name'], labels=True, sticky=True),
    show=False
).add_to(m)




0


In [7]:
# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('Hospital_Scores5.html')